In [367]:
import pandas as pd
import spacy
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet as wn
import spacy
nlp = spacy.load("en_core_web_sm")
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.stem import WordNetLemmatizer 
from collections import defaultdict
import pprint
import argparse
import difflib
from googletrans import Translator
import numpy as np
import pickle
translator = Translator()
import time
from nltk.corpus import wordnet_ic
from gensim.models import KeyedVectors
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import cross_val_score
from nltk.corpus import wordnet as wn
brown_ic = wordnet_ic.ic('ic-brown.dat')
nltk.download('wordnet_ic')
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
import gensim
import os
import collections
import smart_open
import re
import random
from collections import Counter
import spotlight
nltk.download('wordnet')
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords


stop_words = stopwords.words('english')
pd.set_option('display.max_columns', None)  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kadss\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kadss\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kadss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet_ic to
[nltk_data]     C:\Users\kadss\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kadss\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [424]:
#train = pd.read_csv('data/train-set.txt', sep="\t",error_bad_lines=False)
train = train[np.isfinite(train['Gold Tag'])]

In [425]:
train.head()

,id,Sentence1,Sentence2,Gold Tag,tokenized_sent1,tokenized_sent2,lemmatized_sent1,lemmatized_sent2,POS_Tags1,POS_Tags2,Root1,Root2,flagCheckRoot,baseline_sim,word_align_sim,info_con_sim,tag_overlap,verb_diff,noun_diff,adj_diff,adv_diff,mmr,res_sim,no_ques_diff,no_excl_diff,annotations_s1,annotations_s2,num_same_URI,overlap_types,common_entities
0,s_1,But other sources close to the sale said Viven...,But other sources close to the sale said Viven...,4.0,"[But, other, sources, close, to, the, sale, sa...","[But, other, sources, close, to, the, sale, sa...","[But, other, source, close, to, the, sale, sai...","[But, other, source, close, to, the, sale, sai...","[(But, CC), (other, JJ), (source, NN), (close,...","[(But, CC), (other, JJ), (source, NN), (close,...",said,said,True,2.800000,0.740427,4.027374e+00,0.901961,0.333333,0.076923,0.111111,0.333333,"[0.8269078095811264, 0.7994004396802319, 0.499...",2.549887,0,0,{'URI': ['http://dbpedia.org/resource/Vivendi'...,{'URI': ['http://dbpedia.org/resource/Vivendi'...,0,1.000000,0.333333
1,s_2,Micron has declared its first quarterly profit...,Micron's numbers also marked the first quarter...,4.0,"[Micron, has, declared, its, first, quarterly,...","[Micron, 's, numbers, also, marked, the, first...","[Micron, ha, declared, it, first, quarterly, p...","[Micron, 's, number, also, marked, the, first,...","[(Micron, NNP), (ha, NN), (declared, VBD), (it...","[(Micron, NNP), ('s, POS), (number, NN), (also...",declared,marked,False,3.000000,0.790476,5.430591e+00,0.400000,0.333333,0.333333,0.000000,1.000000,"[0.6470795835539085, 1.0, 0.000999000999000999...",2.770720,0,0,"{'URI': [], 'types': []}",{'URI': ['http://dbpedia.org/resource/Micron_T...,0,0.000000,1.000000
2,s_3,The fines are part of failed Republican effort...,"Perry said he backs the Senate's efforts, incl...",3.0,"[The, fines, are, part, of, failed, Republican...","[Perry, said, he, backs, the, Senate, 's, effo...","[The, fine, are, part, of, failed, Republican,...","[Perry, said, he, back, the, Senate, 's, effor...","[(The, DT), (fine, NN), (are, VBP), (part, NN)...","[(Perry, NNP), (said, VBD), (he, PRP), (back, ...",are,said,False,2.000000,0.652066,5.643231e+00,0.625000,0.111111,0.000000,1.000000,0.000000,"[0.8500074996250188, 1000.9999999999999, 0, 1....",2.531463,0,0,{'URI': ['http://dbpedia.org/resource/Republic...,"{'URI': ['http://dbpedia.org/resource/Perry', ...",1,0.333333,0.333333
3,s_4,"The American Anglican Council, which represent...","The American Anglican Council, which represent...",3.0,"[The, American, Anglican, Council, ,, which, r...","[The, American, Anglican, Council, ,, which, r...","[The, American, Anglican, Council, ,, which, r...","[The, American, Anglican, Council, ,, which, r...","[(The, DT), (American, JJ), (Anglican, NNP), (...","[(The, DT), (American, JJ), (Anglican, NNP), (...",said,said,True,4.722222,0.930556,7.816363e+00,0.772727,0.000000,0.285714,0.142857,0.000000,"[0.7000099996666778, 0.7500624843789052, 0, 0....",2.560187,0,0,"{'URI': [], 'types': []}",{'URI': ['http://dbpedia.org/resource/American...,0,0.000000,0.333333
4,s_5,The tech-loaded Nasdaq composite rose 20.96 po...,The technology-laced Nasdaq Composite Index <....,2.0,"[The, tech-loaded, Nasdaq, composite, rose, 20...","[The, technology-laced, Nasdaq, Composite, Ind...","[The, tech-loaded, Nasdaq, composite, rose, 20...","[The, technology-laced, Nasdaq, Composite, Ind...","[(The, DT), (tech-loaded, JJ), (Nasdaq, NNP), ...","[(The, DT), (technology-laced, JJ), (Nasdaq, N...",rose,climbed,False,1.176471,0.562302,1.428571e+299,0.193548,0.333333,0.230769,0.333333,0.000000,"[0.9990009990009991, 0.4999997502498749, 1000....",2.709486,0,0,{'URI': ['http://dbpedia.org/resource/Nasdaq_C...,{'URI': ['http://dbpedia.org/resource/Nasdaq_C...,0,1.000000,0.250000


In [370]:
# Init the Wordnet Lemmatizer
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()

In [371]:
#tokenization
train['tokenized_sent1'] = train.apply(lambda row: nltk.word_tokenize(row['Sentence1']), axis=1)
train['tokenized_sent2'] = train.apply(lambda row: nltk.word_tokenize(row['Sentence2']), axis=1)

In [372]:
#lemmatization
def lemmatize(s):
  
    s = [lemmatizer.lemmatize(word) for word in s]
    return s
    
train['lemmatized_sent1'] = train.apply(lambda row: lemmatize(row['tokenized_sent1']), axis=1)
train['lemmatized_sent2'] = train.apply(lambda row: lemmatize(row['tokenized_sent2']), axis=1)

#stop words
stop = stopwords.words('english')
train['lemmatized_sent1'].apply(lambda x: [item for item in x if item not in stop])
train['lemmatized_sent2'].apply(lambda x: [item for item in x if item not in stop])



0       [But, source, close, sale, said, Vivendi, wa, ...
1       [Micron, 's, number, also, marked, first, quar...
2       [Perry, said, back, Senate, 's, effort, ,, inc...
3       [The, American, Anglican, Council, ,, represen...
4       [The, technology-laced, Nasdaq, Composite, Ind...
5       [Shares, Allergan, 14, cent, $, 78.40, late, t...
6       [More, 130, people, arrested, $, 17, million, ...
7       [Albuquerque, Mayor, Martin, Chavez, said, inv...
8       [The, scientist, also, quarantined, home, soon...
9       [The, upgrade, available, free, download, curr...
10      [A, Cuban, architect, wa, sentenced, 20, year,...
11      [Jim, Williams, ,, director, US-VISIT, project...
12      [The, hearing, came, one, day, Pentagon, first...
13      [In, Nairobi, ,, provost, All, Saints, Cathedr...
14      [Vermillion, ,, Posey, Madison, County, popula...
15      [Swartz, sought, charge, dismissed, ,, saying,...
16       [In, 1995, ,, last, survey, ,, number, equal, .]
17      [The, 

In [373]:
train['POS_Tags1'] = train.apply(lambda row: nltk.pos_tag(row['lemmatized_sent1']), axis=1)
train['POS_Tags2'] = train.apply(lambda row: nltk.pos_tag(row['lemmatized_sent2']), axis=1)

In [374]:
def dependency_parse(sentence):
    doc=nlp(sentence)
    for token in doc:    
        if(token.dep_=="ROOT"):
            return lemmatizer.lemmatize(token.text)

In [375]:
train['Root1'] = train.apply(lambda row: dependency_parse(row['Sentence1']), axis=1)
train['Root2'] = train.apply(lambda row: dependency_parse(row['Sentence2']), axis=1)

In [376]:
train['flagCheckRoot'] = train.apply(lambda row: row['Root1']==row['Root2'], axis=1)

In [377]:
# =========== util func ==============
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'): return 'n'
    if tag.startswith('V'): return 'v'
    if tag.startswith('J'): return 'a'
    if tag.startswith('R'): return 'r'
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None

notin_cnt = [0]
# =========== feature extraction ==============
def sentence_similarity_word_alignment(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet and ppdb """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2)) 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
    score, count = 0.0, 0
    ppdb_score, align_cnt = 0, 0
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        L = [synset.path_similarity(ss) for ss in synsets2]
        L_prime = L
        L = [l for l in L if l]

      

        if L: 
            best_score = max(L)
            score += best_score
            count += 1
    # Average the values
    if count >0: score /= count

    return score



In [378]:
def sentence_similarity_simple_baseline(s1, s2,counts = None):
    def embedding_count(s):
        ret_embedding = defaultdict(int)
        for w in s.split():
            w = w.strip('?.,')
            ret_embedding[w] += 1
        return ret_embedding
    first_sent_embedding = embedding_count(s1)
    second_sent_embedding = embedding_count(s2)
    Embedding1 = []
    Embedding2 = []
    if counts:
        for w in first_sent_embedding:
            Embedding1.append(first_sent_embedding[w] * 1.0/ (counts[w]+0.001))
            Embedding2.append(second_sent_embedding[w] *1.0/ (counts[w]+0.001))
    else:
        for w in first_sent_embedding:
            Embedding1.append(first_sent_embedding[w])
            Embedding2.append(second_sent_embedding[w])
    ret_score = 0
    if not 0 == sum(Embedding2): 
      
        sm= difflib.SequenceMatcher(None,Embedding1,Embedding2)
        ret_score = sm.ratio()*5 
    return ret_score

In [379]:
def extract_overlap_pen(s1, s2):
    """
    :param s1:
    :param s2:
    :return: overlap_pen score
    """
    ss1 = s1.strip().split()
    ss2 = s2.strip().split()
    ovlp_cnt = 0
    for w1 in ss1:
        ovlp_cnt += ss2.count(w1)
    score = 2 * ovlp_cnt / (len(ss1) + len(ss2) + .0)
    return score

In [380]:
def all_token_diff(s1,s2):
    s1, s2 = word_tokenize(s1), word_tokenize(s2)
    pos1, pos2 = pos_tag(s1), pos_tag(s2)
    # all tokens
    return  abs(len(s1) - len(s2)) / float(len(s1) + len(s2))

def adj_diff(s1,s2):
    s1, s2 = word_tokenize(s1), word_tokenize(s2)
    pos1, pos2 = pos_tag(s1), pos_tag(s2)
    # all tokens
    cnt1 = len([1 for item in pos1 if item[1].startswith('J')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('J')])
    if cnt1 == 0 and cnt2 == 0:
        t2 = 0
    else:
        t2 = abs(cnt1 - cnt2) / float(cnt1 + cnt2)
        
    return t2

def adv_diff(s1,s2):
    s1, s2 = word_tokenize(s1), word_tokenize(s2)
    pos1, pos2 = pos_tag(s1), pos_tag(s2)
    cnt1 = len([1 for item in pos1 if item[1].startswith('R')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('R')])
    if cnt1 == 0 and cnt2 == 0:
        t3 = 0
    else:
        t3 = abs(cnt1 - cnt2) / float(cnt1 + cnt2)
        
    return t3


def noun_diff(s1,s2):
    s1, s2 = word_tokenize(s1), word_tokenize(s2)
    pos1, pos2 = pos_tag(s1), pos_tag(s2)
    # all nouns
    cnt1 = len([1 for item in pos1 if item[1].startswith('N')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('N')])
    if cnt1 == 0 and cnt2 == 0:
        t4 = 0
    else:
        t4 = abs(cnt1 - cnt2) / float(cnt1 + cnt2)
        
    return t4

def verb_diff(s1,s2):
    s1, s2 = word_tokenize(s1), word_tokenize(s2)
    pos1, pos2 = pos_tag(s1), pos_tag(s2)
    # all verbs
    cnt1 = len([1 for item in pos1 if item[1].startswith('V')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('V')])
    if cnt1 == 0 and cnt2 == 0:
        t5 = 0
    else:
        t5 = abs(cnt1 - cnt2) / float(cnt1 + cnt2)
    return t5


In [381]:


def extract_mmr_t(s1, s2):
    shorter = 1
    if(len(s1) > len(s2)):  shorter = 2

    s1, s2 = word_tokenize(s1), word_tokenize(s2)
    pos1, pos2 = pos_tag(s1), pos_tag(s2)
    # all tokens
    t1 = (len(s1)+0.001) / (len(s2) +0.001)
    # all adjectives
    cnt1 = len([1 for item in pos1 if item[1].startswith('J')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('J')])
    if cnt1 == 0 and cnt2 == 0:
        t2 = 0
    else:
        t2 = (cnt1 +0.001) / (cnt2 + 0.001)
    # all adverbs
    cnt1 = len([1 for item in pos1 if item[1].startswith('R')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('R')])
    if cnt1 == 0 and cnt2 == 0:
        t3 = 0
    else:
        t3 = (cnt1 +0.001) / (cnt2+0.001)
    # all nouns
    cnt1 = len([1 for item in pos1 if item[1].startswith('N')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('N')])
    if cnt1 == 0 and cnt2 == 0:
        t4 = 0
    else:
        t4 = (cnt1 +0.001) / (cnt2 +0.001)
    # all verbs
    cnt1 = len([1 for item in pos1 if item[1].startswith('V')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('V')])
    if cnt1 == 0 and cnt2 == 0:
        t5 = 0
    else:
        t5 = (cnt1+ 0.001) / (cnt2 + 0.001)

    if shorter == 2:
        t1 = 1 / (t1 + 0.001)
        t2 = 1 / (t2 + 0.001)
        t3 = 1 / (t3 + 0.001)
        t4 = 1 / (t4 + 0.001)
        t5 = 1 / (t5 + 0.001)

    return [t1, t2, t3, t4, t5]    

In [382]:
def sentence_similarity_information_content(sentence1, sentence2):

    ''' compute the sentence similairty using information content from wordnet '''
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2)) 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
    score, count = 0.0, 0
    ppdb_score, align_cnt = 0, 0
    # For each word in the first sentence
    for synset in synsets1:
        L = []
        for ss in synsets2:
            try:
                L.append(synset.res_similarity(ss, brown_ic))
            except:
                continue
        if L: 
            best_score = max(L)
            score += best_score
            count += 1
    # Average the values
    if count >0: score /= count
    return score

In [383]:
vec1 = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)
vec2 = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

def extract_res_vec_similarity(s1, s2):
    first_sents_embeddings = np.empty([0,300])
    second_sents_embeddings = np.empty([0,300])

    first_vecs = np.array([])
    for w in s1.split():
        w = w.strip('?.,')
        if w in vec1:
            first_vec = np.array([vec1[w]])
            if first_vecs.shape[0] == 0:
                first_vecs = first_vec
            else:
                first_vecs = np.vstack((first_vecs, first_vec))
        else:
            if first_vecs.shape[0] == 0:
                first_vecs = np.random.normal(0, 5, 300)
            else:
                first_vecs = np.vstack((first_vecs, np.random.normal(0, 5, 300)))
        # print("first ")
        # print(first_vecs.shape)
    if(first_vecs.shape == (300, )):
        temp = first_vecs
    else:
        temp = np.mean(first_vecs, axis=0)
    # print(temp.shape)
    first_sents_embeddings = np.append(first_sents_embeddings, [temp], axis=0)

    second_vecs = np.array([])  
    for w in s2.split():
        w = w.strip('?.,')
        if w in vec2:
            second_vec = np.array([vec2[w]])
            if second_vecs.shape[0] == 0:
                second_vecs = second_vec
            else:
                second_vecs = np.vstack((second_vecs, second_vec))
        else:
            if second_vecs.shape[0] == 0:
                second_vecs = np.random.normal(0, 5, 300)
            else:
                second_vecs = np.vstack((second_vecs, np.random.normal(0, 5, 300)))
        # print("second ")
        # print(second_vecs.shape)
    if(second_vecs.shape == (300,)):
        temp = second_vecs
    else:
        temp = np.mean(second_vecs, axis=0)
    # print(temp.shape)
    second_sents_embeddings = np.append(second_sents_embeddings, [temp], axis=0)

    for i in range(len(first_sents_embeddings)):
        # cosine similarity

        ret = np.dot(first_sents_embeddings[i], second_sents_embeddings[i]) / (np.linalg.norm(first_sents_embeddings[i]) * np.linalg.norm(second_sents_embeddings[i]))
        ret = 5*(ret + 1) / 2

    return ret

In [384]:
def extract_doc2vec_similarity(s1,s2, model):
    s1 = [w.strip('?.,') for w in s1.split()]
    s2 = [w.strip('?.,') for w in s2.split()]
    embed1 = model.infer_vector(s1)
    embed2 = model.infer_vector(s2)
    ret = np.dot(embed1,embed2)
    return ret

In [385]:
train.head()

,id,Sentence1,Sentence2,Gold Tag,tokenized_sent1,tokenized_sent2,lemmatized_sent1,lemmatized_sent2,POS_Tags1,POS_Tags2,Root1,Root2,flagCheckRoot
0,s_1,But other sources close to the sale said Viven...,But other sources close to the sale said Viven...,4.0,"[But, other, sources, close, to, the, sale, sa...","[But, other, sources, close, to, the, sale, sa...","[But, other, source, close, to, the, sale, sai...","[But, other, source, close, to, the, sale, sai...","[(But, CC), (other, JJ), (source, NN), (close,...","[(But, CC), (other, JJ), (source, NN), (close,...",said,said,True
1,s_2,Micron has declared its first quarterly profit...,Micron's numbers also marked the first quarter...,4.0,"[Micron, has, declared, its, first, quarterly,...","[Micron, 's, numbers, also, marked, the, first...","[Micron, ha, declared, it, first, quarterly, p...","[Micron, 's, number, also, marked, the, first,...","[(Micron, NNP), (ha, NN), (declared, VBD), (it...","[(Micron, NNP), ('s, POS), (number, NN), (also...",declared,marked,False
2,s_3,The fines are part of failed Republican effort...,"Perry said he backs the Senate's efforts, incl...",3.0,"[The, fines, are, part, of, failed, Republican...","[Perry, said, he, backs, the, Senate, 's, effo...","[The, fine, are, part, of, failed, Republican,...","[Perry, said, he, back, the, Senate, 's, effor...","[(The, DT), (fine, NN), (are, VBP), (part, NN)...","[(Perry, NNP), (said, VBD), (he, PRP), (back, ...",are,said,False
3,s_4,"The American Anglican Council, which represent...","The American Anglican Council, which represent...",3.0,"[The, American, Anglican, Council, ,, which, r...","[The, American, Anglican, Council, ,, which, r...","[The, American, Anglican, Council, ,, which, r...","[The, American, Anglican, Council, ,, which, r...","[(The, DT), (American, JJ), (Anglican, NNP), (...","[(The, DT), (American, JJ), (Anglican, NNP), (...",said,said,True
4,s_5,The tech-loaded Nasdaq composite rose 20.96 po...,The technology-laced Nasdaq Composite Index <....,2.0,"[The, tech-loaded, Nasdaq, composite, rose, 20...","[The, technology-laced, Nasdaq, Composite, Ind...","[The, tech-loaded, Nasdaq, composite, rose, 20...","[The, technology-laced, Nasdaq, Composite, Ind...","[(The, DT), (tech-loaded, JJ), (Nasdaq, NNP), ...","[(The, DT), (technology-laced, JJ), (Nasdaq, N...",rose,climbed,False


In [386]:
#Create count dictionary

Counts_for_tf = defaultdict(int)

for sent in train['Sentence1'].tolist():
    for w in [w.strip("?.,") for w in sent.split()]: Counts_for_tf[w] += 1
for sent in train['Sentence2'].tolist():
    for w in [w.strip("?.,") for w in sent.split()]: Counts_for_tf[w] += 1


In [387]:
train['baseline_sim']=train.apply(lambda row: sentence_similarity_simple_baseline(row['Sentence1'],row['Sentence2'],Counts_for_tf), axis=1)

In [473]:
train['word_align_sim']=train.apply(lambda row: sentence_similarity_word_alignment(row['Sentence1'],row['Sentence2']), axis=1)

In [500]:
train['info_con_sim']=train.apply(lambda row: sentence_similarity_information_content(row['Sentence1'],row['Sentence2']), axis=1)

In [390]:
train['tag_overlap']=train.apply(lambda row: extract_overlap_pen(row['Sentence1'],row['Sentence2']), axis=1)

In [391]:
train['verb_diff']=train.apply(lambda row: verb_diff(row['Sentence1'],row['Sentence2']), axis=1)
train['noun_diff']=train.apply(lambda row: noun_diff(row['Sentence1'],row['Sentence2']), axis=1)
train['adj_diff']=train.apply(lambda row: adj_diff(row['Sentence1'],row['Sentence2']), axis=1)
train['adv_diff']=train.apply(lambda row: adv_diff(row['Sentence1'],row['Sentence2']), axis=1)


In [392]:
train['mmr']=train.apply(lambda row: extract_mmr_t(row['Sentence1'],row['Sentence2']), axis=1)

In [393]:
train['res_sim']=train.apply(lambda row: extract_res_vec_similarity(row['Sentence1'],row['Sentence2']), axis=1)

In [394]:
train['no_ques_diff']=train.apply(lambda row: (row['Sentence1'].count("?")-row['Sentence2'].count("?")), axis=1)
train['no_excl_diff']=train.apply(lambda row: (row['Sentence1'].count("!")-row['Sentence2'].count("!")), axis=1)

In [395]:

def duplicates(value1,value2):
    
    dups = Counter(value1) - Counter(value2)
    return len(dups)

In [396]:
def get_annotation(text):
   # print(text)
    try:
        annotations = spotlight.annotate('http://api.dbpedia-spotlight.org/en/annotate',text,
                              confidence=0.4, support=20)
    except Exception as e:
       # print(e)
        return {'URI':[],'types':[]}
    
    URI_lst=[]
    types_lst=[]
    
    for ann_dict in annotations:
        URI_lst.append(ann_dict['URI'])
        types_lst.append(ann_dict['types'])
    
    return {'URI':URI_lst,'types':types_lst}

In [397]:
train['annotations_s1'] = train.apply(lambda row: get_annotation(row['Sentence1']), axis=1)
train['annotations_s2'] = train.apply(lambda row: get_annotation(row['Sentence2']), axis=1)



In [412]:
def get_common_types(sent1_types,sent2_types):
    s1=set(sent1_types)
    s2=set(sent2_types)
    if len(s1.union(s2))>0:
        return len(s1.intersection(s2))/len(s1.union(s2))
    return 0.5

In [413]:
# NER features

train['num_same_URI'] = train.apply(lambda row: duplicates(row['annotations_s1']['URI'],row['annotations_s2']['URI']), axis=1)
# NER features

train['overlap_types'] = train.apply(lambda row:get_common_types(row['annotations_s1']['types'],row['annotations_s2']['types']), axis=1)


In [414]:

def get_common_entities(text1,text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    ent1={X.label_  for X in doc1.ents}
    ent2={X.label_  for X in doc2.ents}
    if len(ent1.union(ent2))>0:
        return len(ent1.intersection(ent2))/len(ent1.union(ent2))
    return 0.5

In [415]:
train['common_entities'] = train.apply(lambda row: get_common_entities(row['Sentence1'],row['Sentence2']), axis=1)


In [416]:
print(train.columns)

Index(['id', 'Sentence1', 'Sentence2', 'Gold Tag', 'tokenized_sent1',
       'tokenized_sent2', 'lemmatized_sent1', 'lemmatized_sent2', 'POS_Tags1',
       'POS_Tags2', 'Root1', 'Root2', 'flagCheckRoot', 'baseline_sim',
       'word_align_sim', 'info_con_sim', 'tag_overlap', 'verb_diff',
       'noun_diff', 'adj_diff', 'adv_diff', 'mmr', 'res_sim', 'no_ques_diff',
       'no_excl_diff', 'annotations_s1', 'annotations_s2', 'num_same_URI',
       'overlap_types', 'common_entities'],
      dtype='object')


In [519]:

X=train[[ 'baseline_sim','Sentence1','Sentence2',
       'word_align_sim', 'info_con_sim', 'tag_overlap', 'res_sim', 'no_ques_diff', 'no_excl_diff', 'num_same_URI', 'overlap_types','common_entities', 'verb_diff', 'noun_diff', 'adj_diff', 'adv_diff']]
Y = train.loc[:, 'Gold Tag']


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)


In [520]:
# create  corpus
all_corpus=train['Sentence1'].tolist()+train['Sentence2'].tolist()

In [521]:
#Pipeline Preprocessing

In [522]:
#classes to return text and numeric values for transformer

from sklearn.base import BaseEstimator, TransformerMixin
class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.field]]
    

In [523]:
def Tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    return lemmatize(words)
from sklearn.svm import SVR, LinearSVR


In [524]:
classifier = Pipeline([
    ('features', FeatureUnion([
        ('tfidf1', Pipeline([
            ('colext', TextSelector('Sentence1')),
            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stop_words,
                     min_df=.0025, max_df=0.25, ngram_range=(1,2))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), 
        ])),
       ('tfidf2', Pipeline([
            ('colext', TextSelector('Sentence2')),
            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stop_words,
                     min_df=.0025, max_df=0.25, ngram_range=(1,2))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), 
        ])),
        ('no_ques_diff', Pipeline([
            ('wordext', NumberSelector('no_ques_diff')),
            ('wscaler', StandardScaler()),
        ])),
         ('no_excl_diff', Pipeline([
            ('wordext', NumberSelector('no_excl_diff')),
            ('wscaler', StandardScaler()),
        ])),
         ('baseline_sim', Pipeline([
            ('wordext', NumberSelector('baseline_sim')),
            ('wscaler', StandardScaler()),
        ])),
         ('word_align_sim', Pipeline([
            ('wordext', NumberSelector('word_align_sim')),
            ('wscaler', StandardScaler()),
        ])),
         ('res_sim', Pipeline([
            ('wordext', NumberSelector('res_sim')),
            ('wscaler', StandardScaler()),
        ])),
         ('num_same_URI', Pipeline([
            ('wordext', NumberSelector('num_same_URI')),
            ('wscaler', StandardScaler()),
        ])),
           ('common_entities', Pipeline([
            ('wordext', NumberSelector('common_entities')),
        
        ])),
           ('verb_diff', Pipeline([
            ('wordext', NumberSelector('verb_diff')),
        
        ])),
         ('noun_diff', Pipeline([
            ('wordext', NumberSelector('noun_diff')),
        
        ])),
        ('adj_diff', Pipeline([
            ('wordext', NumberSelector('adj_diff')),
        
        ])),
        ('adv_diff', Pipeline([
            ('wordext', NumberSelector('adv_diff')),
        
        ])),
         ('overlap_types', Pipeline([
            ('wordext', NumberSelector('overlap_types')),
           
        ])),
           ('tag_overlap', Pipeline([
            ('wordext', NumberSelector('tag_overlap')),
           
        ]))
    ])),
#    ('clf', XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)),
  # ('rf', RandomForestClassifier()),
  ('svr',SVR(kernel='linear')),
     ])

In [525]:

classifier.fit(X_train, y_train)

C:\Users\kadss\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('tfidf1',
                                                 Pipeline(memory=None,
                                                          steps=[('colext',
                                                                  TextSelector(field='Sentence1')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.float64'>,
                                                                                  encoding='utf-8',
            

In [556]:
#preds = classifier.predict(X_test)


preds = [ round(elem) for elem in preds ]

In [557]:
# Evaluation
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score
print("Accuracy:", accuracy_score(y_test, preds))
print(classification_report(y_test, preds))
print(confusion_matrix(y_test, preds))
print("F1-Score:",f1_score(y_test, preds, average='weighted')) 

Accuracy: 0.521978021978022
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         2
         1.0       1.00      0.14      0.25         7
         2.0       0.36      0.25      0.29        20
         3.0       0.40      0.44      0.42        75
         4.0       0.54      0.60      0.57       161
         5.0       0.60      0.53      0.56        99

    accuracy                           0.52       364
   macro avg       0.65      0.49      0.52       364
weighted avg       0.53      0.52      0.52       364

[[ 2  0  0  0  0  0]
 [ 0  1  3  1  2  0]
 [ 0  0  5 10  5  0]
 [ 0  0  3 33 31  8]
 [ 0  0  3 35 97 26]
 [ 0  0  0  3 44 52]]
F1-Score: 0.5183510310383672


In [499]:
from scipy import stats
stats.pearsonr(y_test, preds)

(0.6499291033442438, 4.584129571020399e-45)

In [541]:
key=['p_'+str(i) for i in range(1,len(preds))]
devList =  list(zip(key,preds))
predList =  list(zip(key,y_test))

dev = pd.DataFrame(devList, columns = ['id' , 'Gold Tag']) 
predicted= pd.DataFrame(predList, columns = ['id' , 'Gold Tag']) 

dev['Gold Tag'] = dev['Gold Tag'].astype('str')
#dev = dev.astype(str)
#dev = dev.replace(to_replace = "\.0+$",value = "", regex = True)

#predicted = predicted.astype(str)
#predicted = predicted.replace(to_replace = "\.0+$",value = "", regex = True)

In [550]:
#np.savetxt('dev.txt',dev,delimiter='\t')
#np.savetxt('predicted.txt',predicted,delimiter='\t')

np.savetxt(r'dev.txt', dev.values,fmt='%s')

#dev.to_csv('dev.txt',sep="\t",index=False,dtype=str)

In [554]:
import _pickle as cPickle

# save the classifier
with open('nlp_classifier.pkl', 'wb') as fid:
    cPickle.dump(classifier, fid)    

# load it again
with open('nlp_classifier.pkl', 'rb') as fid:
    gnb_loaded = cPickle.load(fid)

In [558]:
preds = gnb_loaded.predict(X_test)


In [ ]:
import sys
import pandas as pd
import spacy
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet as wn
import spacy
nlp = spacy.load("en_core_web_sm")
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.stem import WordNetLemmatizer 
from collections import defaultdict
import pprint
import argparse
import difflib
from googletrans import Translator
import numpy as np
import pickle
translator = Translator()
import time
from nltk.corpus import wordnet_ic
from gensim.models import KeyedVectors
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import cross_val_score
from nltk.corpus import wordnet as wn
brown_ic = wordnet_ic.ic('ic-brown.dat')
nltk.download('wordnet_ic')
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
import gensim
import os
import collections
import smart_open
import re
import random
from collections import Counter
import spotlight
nltk.download('wordnet')
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from sklearn.svm import SVR, LinearSVR
# Evaluation
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

stop_words = stopwords.words('english')
pd.set_option('display.max_columns', None)  

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()
vec1 = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)
vec2 = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)
stop = stopwords.words('english')


#lemmatization
def lemmatize(s):
  
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

def dependency_parse(sentence):
    doc=nlp(sentence)
    for token in doc:    
        if(token.dep_=="ROOT"):
            return lemmatizer.lemmatize(token.text)
        
# =========== util func ==============
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'): return 'n'
    if tag.startswith('V'): return 'v'
    if tag.startswith('J'): return 'a'
    if tag.startswith('R'): return 'r'
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None

notin_cnt = [0]
# =========== feature extraction ==============
def sentence_similarity_word_alignment(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet and ppdb """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2)) 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
    score, count = 0.0, 0
    ppdb_score, align_cnt = 0, 0
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        L = [synset.path_similarity(ss) for ss in synsets2]
        L_prime = L
        L = [l for l in L if l]

      

        if L: 
            best_score = max(L)
            score += best_score
            count += 1
    # Average the values
    if count >0: score /= count

    return score

def sentence_similarity_simple_baseline(s1, s2,counts = None):
    def embedding_count(s):
        ret_embedding = defaultdict(int)
        for w in s.split():
            w = w.strip('?.,')
            ret_embedding[w] += 1
        return ret_embedding
    first_sent_embedding = embedding_count(s1)
    second_sent_embedding = embedding_count(s2)
    Embedding1 = []
    Embedding2 = []
    if counts:
        for w in first_sent_embedding:
            Embedding1.append(first_sent_embedding[w] * 1.0/ (counts[w]+0.001))
            Embedding2.append(second_sent_embedding[w] *1.0/ (counts[w]+0.001))
    else:
        for w in first_sent_embedding:
            Embedding1.append(first_sent_embedding[w])
            Embedding2.append(second_sent_embedding[w])
    ret_score = 0
    if not 0 == sum(Embedding2): 
      
        sm= difflib.SequenceMatcher(None,Embedding1,Embedding2)
        ret_score = sm.ratio()*5 
    return ret_score

def extract_overlap_pen(s1, s2):
    """
    :param s1:
    :param s2:
    :return: overlap_pen score
    """
    ss1 = s1.strip().split()
    ss2 = s2.strip().split()
    ovlp_cnt = 0
    for w1 in ss1:
        ovlp_cnt += ss2.count(w1)
    score = 2 * ovlp_cnt / (len(ss1) + len(ss2) + .0)
    return score

def all_token_diff(s1,s2):
    s1, s2 = word_tokenize(s1), word_tokenize(s2)
    pos1, pos2 = pos_tag(s1), pos_tag(s2)
    # all tokens
    return  abs(len(s1) - len(s2)) / float(len(s1) + len(s2))

def adj_diff(s1,s2):
    s1, s2 = word_tokenize(s1), word_tokenize(s2)
    pos1, pos2 = pos_tag(s1), pos_tag(s2)
    # all tokens
    cnt1 = len([1 for item in pos1 if item[1].startswith('J')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('J')])
    if cnt1 == 0 and cnt2 == 0:
        t2 = 0
    else:
        t2 = abs(cnt1 - cnt2) / float(cnt1 + cnt2)
        
    return t2

def adv_diff(s1,s2):
    s1, s2 = word_tokenize(s1), word_tokenize(s2)
    pos1, pos2 = pos_tag(s1), pos_tag(s2)
    cnt1 = len([1 for item in pos1 if item[1].startswith('R')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('R')])
    if cnt1 == 0 and cnt2 == 0:
        t3 = 0
    else:
        t3 = abs(cnt1 - cnt2) / float(cnt1 + cnt2)
        
    return t3


def noun_diff(s1,s2):
    s1, s2 = word_tokenize(s1), word_tokenize(s2)
    pos1, pos2 = pos_tag(s1), pos_tag(s2)
    # all nouns
    cnt1 = len([1 for item in pos1 if item[1].startswith('N')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('N')])
    if cnt1 == 0 and cnt2 == 0:
        t4 = 0
    else:
        t4 = abs(cnt1 - cnt2) / float(cnt1 + cnt2)
        
    return t4

def verb_diff(s1,s2):
    s1, s2 = word_tokenize(s1), word_tokenize(s2)
    pos1, pos2 = pos_tag(s1), pos_tag(s2)
    # all verbs
    cnt1 = len([1 for item in pos1 if item[1].startswith('V')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('V')])
    if cnt1 == 0 and cnt2 == 0:
        t5 = 0
    else:
        t5 = abs(cnt1 - cnt2) / float(cnt1 + cnt2)
    return t5

def sentence_similarity_information_content(sentence1, sentence2):

    ''' compute the sentence similairty using information content from wordnet '''
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2)) 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
    score, count = 0.0, 0
    ppdb_score, align_cnt = 0, 0
    # For each word in the first sentence
    for synset in synsets1:
        L = []
        for ss in synsets2:
            try:
                L.append(synset.res_similarity(ss, brown_ic))
            except:
                continue
        if L: 
            best_score = max(L)
            score += best_score
            count += 1
    # Average the values
    if count >0: score /= count
    return score


def extract_res_vec_similarity(s1, s2):
    first_sents_embeddings = np.empty([0,300])
    second_sents_embeddings = np.empty([0,300])

    first_vecs = np.array([])
    for w in s1.split():
        w = w.strip('?.,')
        if w in vec1:
            first_vec = np.array([vec1[w]])
            if first_vecs.shape[0] == 0:
                first_vecs = first_vec
            else:
                first_vecs = np.vstack((first_vecs, first_vec))
        else:
            if first_vecs.shape[0] == 0:
                first_vecs = np.random.normal(0, 5, 300)
            else:
                first_vecs = np.vstack((first_vecs, np.random.normal(0, 5, 300)))
        # print("first ")
        # print(first_vecs.shape)
    if(first_vecs.shape == (300, )):
        temp = first_vecs
    else:
        temp = np.mean(first_vecs, axis=0)
    # print(temp.shape)
    first_sents_embeddings = np.append(first_sents_embeddings, [temp], axis=0)

    second_vecs = np.array([])  
    for w in s2.split():
        w = w.strip('?.,')
        if w in vec2:
            second_vec = np.array([vec2[w]])
            if second_vecs.shape[0] == 0:
                second_vecs = second_vec
            else:
                second_vecs = np.vstack((second_vecs, second_vec))
        else:
            if second_vecs.shape[0] == 0:
                second_vecs = np.random.normal(0, 5, 300)
            else:
                second_vecs = np.vstack((second_vecs, np.random.normal(0, 5, 300)))
        # print("second ")
        # print(second_vecs.shape)
    if(second_vecs.shape == (300,)):
        temp = second_vecs
    else:
        temp = np.mean(second_vecs, axis=0)
    # print(temp.shape)
    second_sents_embeddings = np.append(second_sents_embeddings, [temp], axis=0)

    for i in range(len(first_sents_embeddings)):
        # cosine similarity

        ret = np.dot(first_sents_embeddings[i], second_sents_embeddings[i]) / (np.linalg.norm(first_sents_embeddings[i]) * np.linalg.norm(second_sents_embeddings[i]))
        ret = 5*(ret + 1) / 2

    return ret


def extract_doc2vec_similarity(s1,s2, model):
    s1 = [w.strip('?.,') for w in s1.split()]
    s2 = [w.strip('?.,') for w in s2.split()]
    embed1 = model.infer_vector(s1)
    embed2 = model.infer_vector(s2)
    ret = np.dot(embed1,embed2)
    return ret



def duplicates(value1,value2):
    
    dups = Counter(value1) - Counter(value2)
    return len(dups)

def get_annotation(text):
   # print(text)
    try:
        annotations = spotlight.annotate('http://api.dbpedia-spotlight.org/en/annotate',text,
                              confidence=0.4, support=20)
    except Exception as e:
       # print(e)
        return {'URI':[],'types':[]}
    
    URI_lst=[]
    types_lst=[]
    
    for ann_dict in annotations:
        URI_lst.append(ann_dict['URI'])
        types_lst.append(ann_dict['types'])
    
    return {'URI':URI_lst,'types':types_lst}

def get_common_types(sent1_types,sent2_types):
    s1=set(sent1_types)
    s2=set(sent2_types)
    if len(s1.union(s2))>0:
        return len(s1.intersection(s2))/len(s1.union(s2))
    return 0.5



def get_common_entities(text1,text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    ent1={X.label_  for X in doc1.ents}
    ent2={X.label_  for X in doc2.ents}
    if len(ent1.union(ent2))>0:
        return len(ent1.intersection(ent2))/len(ent1.union(ent2))
    return 0.5

#classes to return text and numeric values for transformer

from sklearn.base import BaseEstimator, TransformerMixin
class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.field]]
    
    
def Tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    return lemmatize(words)

def build_features(train):
    
    Counts_for_tf = defaultdict(int)
    
    for sent in train['Sentence1'].tolist():
        for w in [w.strip("?.,") for w in sent.split()]: Counts_for_tf[w] += 1
    for sent in train['Sentence2'].tolist():
        for w in [w.strip("?.,") for w in sent.split()]: Counts_for_tf[w] += 1
        
        
    

    #tokenization    
    train['tokenized_sent1'] = train.apply(lambda row: nltk.word_tokenize(row['Sentence1']), axis=1)
    train['tokenized_sent2'] = train.apply(lambda row: nltk.word_tokenize(row['Sentence2']), axis=1)
    
    #lemmatization
    train['lemmatized_sent1'] = train.apply(lambda row: lemmatize(row['tokenized_sent1']), axis=1)
    train['lemmatized_sent2'] = train.apply(lambda row: lemmatize(row['tokenized_sent2']), axis=1)
    
    #stop words removal    
    train['lemmatized_sent1'].apply(lambda x: [item for item in x if item not in stop])
    train['lemmatized_sent2'].apply(lambda x: [item for item in x if item not in stop])

    #POS tags    
    train['POS_Tags1'] = train.apply(lambda row: nltk.pos_tag(row['lemmatized_sent1']), axis=1)
    train['POS_Tags2'] = train.apply(lambda row: nltk.pos_tag(row['lemmatized_sent2']), axis=1)
    
    train['Root1'] = train.apply(lambda row: dependency_parse(row['Sentence1']), axis=1)
    train['Root2'] = train.apply(lambda row: dependency_parse(row['Sentence2']), axis=1)
    
    train['flagCheckRoot'] = train.apply(lambda row: row['Root1']==row['Root2'], axis=1)
    
    train['baseline_sim']=train.apply(lambda row: sentence_similarity_simple_baseline(row['Sentence1'],row['Sentence2'],Counts_for_tf), axis=1)
    train['word_align_sim']=train.apply(lambda row: sentence_similarity_word_alignment(row['Sentence1'],row['Sentence2']), axis=1)
    
    train['baseline_sim']=train.apply(lambda row: sentence_similarity_simple_baseline(row['Sentence1'],row['Sentence2'],Counts_for_tf), axis=1)
    train['word_align_sim']=train.apply(lambda row: sentence_similarity_word_alignment(row['Sentence1'],row['Sentence2']), axis=1)
    train['info_con_sim']=train.apply(lambda row: sentence_similarity_information_content(row['Sentence1'],row['Sentence2']), axis=1)
    train['tag_overlap']=train.apply(lambda row: extract_overlap_pen(row['Sentence1'],row['Sentence2']), axis=1)
    
    train['verb_diff']=train.apply(lambda row: verb_diff(row['Sentence1'],row['Sentence2']), axis=1)
    train['noun_diff']=train.apply(lambda row: noun_diff(row['Sentence1'],row['Sentence2']), axis=1)
    train['adj_diff']=train.apply(lambda row: adj_diff(row['Sentence1'],row['Sentence2']), axis=1)
    train['adv_diff']=train.apply(lambda row: adv_diff(row['Sentence1'],row['Sentence2']), axis=1)
    
    
    train['mmr']=train.apply(lambda row: extract_mmr_t(row['Sentence1'],row['Sentence2']), axis=1)
    train['res_sim']=train.apply(lambda row: extract_res_vec_similarity(row['Sentence1'],row['Sentence2']), axis=1)
    
    train['no_ques_diff']=train.apply(lambda row: (row['Sentence1'].count("?")-row['Sentence2'].count("?")), axis=1)
    train['no_excl_diff']=train.apply(lambda row: (row['Sentence1'].count("!")-row['Sentence2'].count("!")), axis=1)
    
    
    train['annotations_s1'] = train.apply(lambda row: get_annotation(row['Sentence1']), axis=1)
    train['annotations_s2'] = train.apply(lambda row: get_annotation(row['Sentence2']), axis=1)
    
    
    # NER features

    train['num_same_URI'] = train.apply(lambda row: duplicates(row['annotations_s1']['URI'],row['annotations_s2']['URI']), axis=1)
    # NER features

    train['overlap_types'] = train.apply(lambda row:get_common_types(row['annotations_s1']['types'],row['annotations_s2']['types']), axis=1)
    
    train['common_entities'] = train.apply(lambda row: get_common_entities(row['Sentence1'],row['Sentence2']), axis=1)




    
    

if __name__ == "__main__":
    #train_file=sys.argv[0]
    train_file='data/train-set.txt'
    train = pd.read_csv(train_file, sep="\t",error_bad_lines=False)
    train = train[np.isfinite(train['Gold Tag'])]  #remove nan values
    
    processed_train=build_features(train)
    
    X=processed_train[[ 'baseline_sim','Sentence1','Sentence2',
       'word_align_sim', 'tag_overlap', 'res_sim', 'no_ques_diff', 'no_excl_diff', 'num_same_URI', 'overlap_types','common_entities', 'verb_diff', 'noun_diff', 'adj_diff', 'adv_diff']]
    Y = processed_train.loc[:, 'Gold Tag']
    
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
    
    all_corpus=train['Sentence1'].tolist()+train['Sentence2'].tolist()
    
    classifier = Pipeline([
    ('features', FeatureUnion([
        ('tfidf1', Pipeline([
            ('colext', TextSelector('Sentence1')),
            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stop_words,
                     min_df=.0025, max_df=0.25, ngram_range=(1,2))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), 
        ])),
       ('tfidf2', Pipeline([
            ('colext', TextSelector('Sentence2')),
            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stop_words,
                     min_df=.0025, max_df=0.25, ngram_range=(1,2))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), 
        ])),
        ('no_ques_diff', Pipeline([
            ('wordext', NumberSelector('no_ques_diff')),
            ('wscaler', StandardScaler()),
        ])),
         ('no_excl_diff', Pipeline([
            ('wordext', NumberSelector('no_excl_diff')),
            ('wscaler', StandardScaler()),
        ])),
         ('baseline_sim', Pipeline([
            ('wordext', NumberSelector('baseline_sim')),
            ('wscaler', StandardScaler()),
        ])),
         ('word_align_sim', Pipeline([
            ('wordext', NumberSelector('word_align_sim')),
            ('wscaler', StandardScaler()),
        ])),
         ('res_sim', Pipeline([
            ('wordext', NumberSelector('res_sim')),
            ('wscaler', StandardScaler()),
        ])),
         ('num_same_URI', Pipeline([
            ('wordext', NumberSelector('num_same_URI')),
            ('wscaler', StandardScaler()),
        ])),
         ('info_con_sim', Pipeline([
            ('wordext', NumberSelector('info_con_sim')),
        
        ])),
           ('common_entities', Pipeline([
            ('wordext', NumberSelector('common_entities')),
        
        ])),
           ('verb_diff', Pipeline([
            ('wordext', NumberSelector('verb_diff')),
        
        ])),
         ('noun_diff', Pipeline([
            ('wordext', NumberSelector('noun_diff')),
        
        ])),
        ('adj_diff', Pipeline([
            ('wordext', NumberSelector('adj_diff')),
        
        ])),
        ('adv_diff', Pipeline([
            ('wordext', NumberSelector('adv_diff')),
        
        ])),
         ('overlap_types', Pipeline([
            ('wordext', NumberSelector('overlap_types')),
           
        ])),
           ('tag_overlap', Pipeline([
            ('wordext', NumberSelector('tag_overlap')),
           
        ]))
    ])),
#    ('clf', XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)),
   ('rf', RandomForestClassifier()),
  #('svr',SVR(kernel='linear')),
     ])
            

    
    classifier.fit(X_train, y_train) 
    # save the classifier
with open('nlp_classifier.pkl', 'wb') as fid:
    cPickle.dump(classifier, fid)    

    preds = classifier.predict(X_test)
    
    return y_test,preds
            